<a href="https://colab.research.google.com/github/HappyJJins/KakaoArena/blob/master/TEST02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive, auth

ROOT = '/content/drive'
drive.mount(ROOT)

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

In [ ]:
!git clone https://github.com/HappyJJins/Jeong-Hajin.git

In [ ]:
!ls -ltr

In [ ]:
!pip install scikit-surprise
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from wordcloud import WordCloud
font_path = '/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/NanumGothic.ttf'

import warnings; warnings.simplefilter('ignore')

In [ ]:
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re

import numpy as np
import pandas as pd
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from konlpy.tag import Twitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from pandas.plotting import register_matplotlib_converters
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [ ]:
# playlist에 수록된 tags와 songs의 일부분만 들어있는 val파일
# algorithm test용 파일
val = pd.read_json('/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/data/val.json', typ='frame')
val.info()

In [ ]:
val.head()

In [ ]:
# playlist별 수록된 모든 tags, songs가 들어있는 train파일
train = pd.read_json('/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/data/train.json', typ = 'frame')
train.info()

In [ ]:
train.head()

In [ ]:
# melon에 있는 모든 songs 정보가 담긴 song_meta파일
song_meta = pd.read_json('/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/data/song_meta.json', typ = 'frame')
song_meta.info()

In [ ]:
song_meta.head()

In [ ]:
# playlist id와 수록된 songs 추출
# playlist id별 수록된 song list 의 형태
plylst_song_map = train[['id', 'songs']]

# unnest songs
plylst_song_map_unnest = np.dstack(
    (
        # playlist에 수록된 song 갯수만큼 playlist id 반복
        # playlist id와 수록된 song을 1:1 매칭
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))),
        np.concatenate(plylst_song_map.songs.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# unnest 객체 제거
del plylst_song_map_unnest

In [ ]:
plylst_song_map

In [ ]:
# song id와 대분류 장르코드 리스트(song_gn_gnr_basket) 추출
song_gnr_map = song_meta.loc[:, ['id', 'song_gn_gnr_basket']]

# unnest song_gn_gnr_basket
song_gnr_map_unnest = np.dstack(
    (
        # song id를 매핑된 genre수 만큼 반복
        # song id와 genre 1:1 매칭
        np.repeat(song_gnr_map.id.values, list(map(len, song_gnr_map.song_gn_gnr_basket))), 
        np.concatenate(song_gnr_map.song_gn_gnr_basket.values)
    )
)

# unnested 데이터프레임 생성 : song_gnr_map
song_gnr_map = pd.DataFrame(data = song_gnr_map_unnest[0], columns = song_gnr_map.columns)
song_gnr_map['id'] = song_gnr_map['id'].astype(str)
song_gnr_map.rename(columns = {'id' : 'song_id', 'song_gn_gnr_basket' : 'gnr_code'}, inplace = True)

# unnest 객체 제거
del song_gnr_map_unnest

In [ ]:
song_gnr_map.head()

In [ ]:
# playlist id와 매핑된 tags 추출
plylst_tag_map = train[['id', 'tags']]

# unnest tags
plylst_tag_map_unnest = np.dstack(
    (
        # playlist id를 매핑된 tags 수 만큼 반복
        # playlist id와 tag 1:1 매칭
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))), 
        np.concatenate(plylst_tag_map.tags.values)
    )
)

# unnested 데이터프레임 생성 : plylst_tag_map
plylst_tag_map = pd.DataFrame(data = plylst_tag_map_unnest[0], columns = plylst_tag_map.columns)
plylst_tag_map['id'] = plylst_tag_map['id'].astype(str)

# unnest 객체 제거
del plylst_tag_map_unnest

In [ ]:
plylst_tag_map.head()

In [ ]:
# 플레이리스트-곡 테이블 plylst_song_map + 플레이리스트-태그 테이블 plylst_tag_map join 테이블 생성 : plylst_song_tag_map
plylst_song_tag_map = pd.merge(plylst_song_map, plylst_tag_map, how = 'left', on = 'id')

# 1번 테이블 plylst_song_tag_map + 곡 장르 테이블 song_gnr_map join
# playlist_id + song_id + tag + genre -> 1:1:1:1 매칭
plylst_song_tag_map = pd.merge(plylst_song_tag_map, song_gnr_map, how = 'left', left_on = 'songs', right_on = 'song_id')
plylst_song_tag_map.head()

In [ ]:
# 태그별 매핑된 장르 리스트
tag_gnr = pd.DataFrame()
tag_gnr['gnr_code'] = plylst_song_tag_map.groupby('tags').gnr_code.unique()
tag_gnr['gnr_code'] = tag_gnr['gnr_code'].apply(lambda x: x.tolist())
tag_gnr.reset_index(level=['tags'], inplace = True)
tag_gnr.head()

In [ ]:
# test용으로 val파일을 복제한 val_train 파일 생성
val_train = val
# song_id와 plylst_Id type을 int에서 str으로 변환
val_train['songs'] = val_train['songs'].apply(lambda x: list(map(str, x)))
val_train['id'] = val_train['id'].astype(str)

In [ ]:
# bad algorithm code. 추후 수정 필요
def test_algo(id):
  # 입력된 playlist_id에 매핑된 tags와 songs list 저장
  tags = val_train[val_train['id'] == id]['tags'].tolist()[0]
  songs = val_train[val_train['id'] == id]['songs'].tolist()[0]

  # 매핑된 tags와 songs의 장르 저장
  tags_gnr = list(map(lambda x: tag_gnr[tag_gnr['tags'] == x]['gnr_code'].tolist()[0], tags))
  songs_gnr = list(map(lambda x: song_gnr_map[song_gnr_map['song_id'] == x]['gnr_code'].tolist()[0], songs))

  # 중복되는 genre 제거
  test = []
  for x in tags_gnr:
    test +=x
  test += list(set(songs_gnr))
  gnr = list(set(test))
  return gnr

In [ ]:
test_algo('122127')